In [1]:
import pandas as pd
import requests
from io import StringIO


def _get_url(site, param_cd, start_dt, end_dt):
    url_base = 'https://waterservices.usgs.gov/nwis/iv/'
    url_params_ind = '?'
    url_params = f'sites={site}&parameterCd={param_cd}&startDT={start_dt}&endDT={end_dt}&siteStatus=all&format=rdb'
    
    return url_base + url_params_ind + url_params


def _clean_text_data(text):
    clean_tsv = ''
    for line in text.splitlines():
        if not line.startswith('#'):
            clean_tsv += line + '\n'

    return clean_tsv


def _read_usgs_tsv(tsv):
    
    return pd.read_csv(StringIO(tsv), sep='\t', lineterminator='\n', skiprows=[1])


def _verify(text):
    status = True
    
    if 'The request sent by the client was syntactically incorrect' in text:
        print('A parameter was likely mistyped.')
        status = False
        
    elif 'No sites found matching' in text:
        print('The site doesn\'t measure the provided parameter.')
        status = False
    
    return status


def _get_param(param_cd):
    param_dict = {
        '00010' : 'temperature',
        '00050' : 'height'
    }
    
    return param_dict[param_cd]


def _transform_df(df, param_cd):
    for c in df.columns:
        if re.search('\d+_\d+_cd', c):
            drop_col = re.match('\d+_\d+_cd', c).string
            usgs_water_data = usgs_water_data.drop(drop_col, axis = 1)
        elif re.search('\d+_\d+$', c):
            rnm_col = re.match('\d+_\d+$', c).string    


def get_usgs_water_data(site, param_cd, start_dt, end_dt):
    r = requests.get(_get_url(site, param_cd, start_dt, end_dt))
    raw_tsv = r.text
    if _verify(raw_tsv):
        cleaned_tsv = _clean_text_data(raw_tsv)
        df = _read_usgs_tsv(cleaned_tsv)
    else:
        df = None
    
    return df

In [3]:
site = '02055080'
param_cd = '00010'
start_dt = '2023-02-20T19:17:43.422-05:00'
end_dt = '2023-02-27T19:17:43.422-05:00'

r = requests.get(_get_url(site, param_cd, start_dt, end_dt))
'No sites found matching' in r.text

get_usgs_water_data(site, param_cd, start_dt, end_dt)

,agency_cd,site_no,datetime,tz_cd,147479_00010,147479_00010_cd
0,USGS,2055080,2023-02-20 19:30,EST,9.7,P
1,USGS,2055080,2023-02-20 19:45,EST,9.7,P
2,USGS,2055080,2023-02-20 20:00,EST,9.7,P
3,USGS,2055080,2023-02-20 20:15,EST,9.7,P
4,USGS,2055080,2023-02-20 20:30,EST,9.7,P
...,...,...,...,...,...,...
667,USGS,2055080,2023-02-27 18:15,EST,10.2,P
668,USGS,2055080,2023-02-27 18:30,EST,10.2,P
669,USGS,2055080,2023-02-27 18:45,EST,10.2,P
670,USGS,2055080,2023-02-27 19:00,EST,10.2,P
